# Projet 5 : Segmenter des clients d'un site e-commerce

*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *Data Scientist* d'OpenClassroooms.

L'objectif pricipal est de réaliser **une segmentation des clients** d'un site de e-commerce, **une proposition de contrat de maintenance** devra être inclue.

Les données mises à notre disposition proviennent du site *kaggle* :
https://www.kaggle.com/olistbr/brazilian-ecommerce

# Partie I : Data Wrangling

L'objectif de ce notebook est de décrire les opérations de nettoyage nécessaires à l'obtention d'un jeu de données exploitable.

In [1]:
# Import usual libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Récupération-des-données" data-toc-modified-id="Récupération-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Récupération des données</a></span></li><li><span><a href="#Fusion-des-données" data-toc-modified-id="Fusion-des-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fusion des données</a></span></li><li><span><a href="#Ingénierie-des-variables" data-toc-modified-id="Ingénierie-des-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ingénierie des variables</a></span><ul class="toc-item"><li><span><a href="#Variables-RFM" data-toc-modified-id="Variables-RFM-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Variables RFM</a></span></li><li><span><a href="#Variables-liées-aux-produits" data-toc-modified-id="Variables-liées-aux-produits-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Variables liées aux produits</a></span></li><li><span><a href="#Catégories-des-produits" data-toc-modified-id="Catégories-des-produits-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Catégories des produits</a></span></li><li><span><a href="#Variables-liées-aux-commentaires" data-toc-modified-id="Variables-liées-aux-commentaires-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Variables liées aux commentaires</a></span></li><li><span><a href="#Moyens-de-paiement" data-toc-modified-id="Moyens-de-paiement-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Moyens de paiement</a></span></li><li><span><a href="#Les-plus-gros-vendeurs" data-toc-modified-id="Les-plus-gros-vendeurs-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Les plus gros vendeurs</a></span></li><li><span><a href="#États" data-toc-modified-id="États-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>États</a></span></li><li><span><a href="#Fusion-des-pivot-tables" data-toc-modified-id="Fusion-des-pivot-tables-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Fusion des pivot tables</a></span></li></ul></li><li><span><a href="#Export-des-données" data-toc-modified-id="Export-des-données-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export des données</a></span></li></ul></div>

## Récupération des données

Une fois les données téléchargées, nous pouvons les charger dans un DataFrame en utilisant la librairie **pandas**.

In [3]:
df_customers =  pd.read_csv("data/olist_customers_dataset.csv")
df_geolocation = pd.read_csv("data/olist_geolocation_dataset.csv")
df_order_items = pd.read_csv("data/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv("data/olist_order_payments_dataset.csv")
df_order_reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv("data/olist_orders_dataset.csv")
df_products = pd.read_csv("data/olist_products_dataset.csv")
df_sellers = pd.read_csv("data/olist_sellers_dataset.csv")
df_translation = pd.read_csv("data/product_category_name_translation.csv")

Regardons ce qui est contenu dans chaque DataFrame.

In [4]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [5]:
df_geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [6]:
df_order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [7]:
df_order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [8]:
df_order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                100000 non-null  object
 1   order_id                 100000 non-null  object
 2   review_score             100000 non-null  int64 
 3   review_comment_title     11715 non-null   object
 4   review_comment_message   41753 non-null   object
 5   review_creation_date     100000 non-null  object
 6   review_answer_timestamp  100000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [9]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [10]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [11]:
df_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [12]:
df_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


Nous pouvons noter plusieurs choses :
1. Il va falloir fusionner ces tables afin de créer une table unique
2. La table geolocation n'apporte rien de plus que la table *customers*
3. Beaucoup de données redondantes
4. Les tables contiennent très peu de données manquantes

## Fusion des données

Avant de commencer à nettoyer les données, nous allons commencer par fusionner les données afin d'obtenir une table unique. Ce qui sera bien plus pratique pour manipuler les données.  
Nous en profiterons pour convertir les dates en utilant une base mensuelle.

In [22]:
def merge_data(list_df):
    """Function merging different DataFrames to get a unique one.
     Parameters:
    list_df: list object
        a list with all dataframes to be merged
    -----------
    Return:
        DataFrame
    """
    df = (pd.merge(left=list_df[0], right=list_df[1],
                   how="left", on="customer_id")
            .merge(right=list_df[2], how="left",
                   on="order_id", copy=False)
            .merge(right=list_df[3], how="left",
                   on="product_id", copy=False)
            .merge(right=list_df[4], how="left",
                   on="order_id", copy=False)
            .merge(right=list_df[5], how="left",
                   on="order_id", copy=False))
    return df

In [54]:
list_df = [df_customers,
           df_orders,
           df_order_items,
           df_products,
           df_order_payments,
           df_order_reviews]

df = merge_data(list_df)

In [56]:
def date_monthly(data, feat_to_convert):
    """Function converting data features into a monthy basis date.
    Parameters:
    data: DataFrame
        the pandas object holding data
    feat_to_convert: list of strings
        list holding the name of features to convert
    -----------
    Return:
        DataFrame
    """
    for c in feat_to_convert:
        data[c] = pd.to_datetime(data[c]).dt.date
        #data[c] = pd.to_datetime(data[c].dt.strftime('%Y-%m'))
    return data

In [57]:
feat_to_convert = ["order_purchase_timestamp"]
df = date_monthly(df, feat_to_convert)

## Ingénierie des variables

Bon nombre des variables ne sont pas exploitables en l'état, il va falloir les transformer avant de penser à les utiliser pour faire tourner des modèles de partionnement.

De plus, ayant pour but de faire une segmentation des clients il va nous falloir regrouper les données par client. Dans la base de donnée, un client unique est assigné à chaque transaction. Ainsi, un client ayant effectué plusieurs transactions se verra attribuer plusieurs *customer_id*. C'est pourquoi nous avons aussi accès à la variable *customer_unique_id*. C'est cette dernière que nous utiliserons pour regrouper les données.

### Variables RFM

La méthode de segmentation de clients la plus utilisée est le calcul d'un score RFM (Recency, Frequency, Monetary Value). Ce score consiste à donner une note entre 1 et 10 pour trois variables :
- récence : maximum entre "10 - le nombre de mois écoulés entre aujourd'hui et le dernier achat effectué" et 1
- fréquence : maximum entre "le nombre d'achats effectués (avec une limite de 10) sur une période (en général 12 mois) et 1".
- achat : totalité des achats sur une période de temps ou montant du panier moyen. Ici nous prendrons le panier moyen, et attribuerons comme note le numéro du décile.

Le score est obtenu en additionnant ces trois notes.

In [105]:
def get_rfm(data, m_mean=False):
    """Function to get the three RFM features,
    commonly used in database marketing.
    Parameters:
    data: DataFrame
        the pandas object holding data
    m_mean: bool, default False
        to get the mean and not the total for the monetary value
    -----------
    Return:
        DataFrame
    """
    r_feature = "order_purchase_timestamp"
    f_feature = "order_id"
    m_feature = "price"
    values = [r_feature, f_feature, m_feature]
    n_customers = data["customer_unique_id"].nunique()
    # Set a reference date
    date_ref = data[r_feature].max()
    # define the aggregating function for each feature
    aggfunc = {r_feature: max,
               f_feature: pd.Series.nunique,
               m_feature: sum}
    # group by unique client
    df = pd.pivot_table(data, index="customer_unique_id",
                        values=values, aggfunc=aggfunc)
    # rename features
    df = df.rename(columns={r_feature: "Recency",
                            f_feature: "Frequency",
                            m_feature: "Monetary_value"})
    # get the score for the Recency
    df["Recency"] = 10 - (date_ref-df["Recency"])/np.timedelta64(1, 'M')
    df["Recency"].round().fillna(0, inplace=True)
    df["Recency"].where(df["Recency"]>0, other=1, inplace=True)
    # get the score for Moneaty_value
    df["Monetary_value"].fillna(0, inplace=True)
    if m_mean:
        df["Monetary_value"] /= df["Frequency"]
    df["Monetary_value"] = (pd.qcut(df["Monetary_value"], q=10,
                                   labels=np.linspace(1, 10, 10))
                              .astype(int))
    # get the score for the Frequency
    df["Frequency"].fillna(0, inplace=True)
    df["Frequency"].where(df["Frequency"]>0, other=1, inplace=True)
    df["Frequency"].where(df["Frequency"]<10, other=10, inplace=True)
    return df[["Recency", "Frequency", "Monetary_value"]]

In [106]:
data = get_rfm(df, m_mean=True)
data.describe()

,Recency,Frequency,Monetary_value
count,96096.000000,96096.000000,96096.000000
mean,2.827508,1.034736,5.491363
std,2.388015,0.210106,2.870511
min,0.012115,1.000000,1.000000
25%,1.000000,1.000000,3.000000
50%,1.162036,1.000000,5.000000
75%,4.611799,1.000000,8.000000
max,10.000000,10.000000,10.000000


In [ ]:
# Create a new dataframe with all required data
df = pd.merge(df_orders, df_order_items, how="inner",
              on="order_id", copy=True)
df = df.merge(right=df_customers, how="inner",
                on="customer_id", copy=False)

values = ["order_id",
          "product_id",
          "price",
          "freight_value",
          "order_status"]
aggfunc = {"order_id": pd.Series.nunique,
           "product_id": "count",
           "price": sum,
           "freight_value": sum,
           "order_status": lambda s: s[s=="delivered"].count()}

pivot_orders = (pd.pivot_table(df, index="customer_unique_id",
                               values=values, aggfunc=aggfunc)
                  .rename(columns={"order_id": "n_orders",
                                   "product_id": "order_n_products",
                                   "price": "order_price",
                                   "freight_value": "order_freight_value",
                                   "order_status": "order_delivered"})
                  .assign(order_delivered=lambda df: df[df["order_n_products"]>0]["order_delivered"]/
                                                     df[df["order_n_products"]>0]["order_n_products"],               
                          order_n_products=lambda df: df["order_n_products"]/df["n_orders"],
                          order_price=lambda df: df["order_price"]/df["n_orders"],
                          order_freight_value= lambda df: df["order_freight_value"]/df["n_orders"]))

In [ ]:
pivot_orders.describe()

### Variables liées aux produits

Nous traiterons ensuite les variables liées aux produits. Pour chaque client, nous calculerons les variables suivantes :
- prix moyen par produit
- nombre moyen de photos par produit
- longeur moyenne de description par produit
- frais de port moyens par produit

In [ ]:
# Add products data
df = df.merge(right=df_products, how="inner",
                  on="product_id", copy=False)

values = ["price",
          "product_photos_qty",
          "product_description_lenght",
          "freight_value"]
aggfunc = {"price": 'mean',
           "product_photos_qty": 'mean',
           "product_description_lenght": 'mean',
           "freight_value": 'mean'}

pivot_products = pd.pivot_table(df, index="customer_unique_id",
                                values=values, aggfunc=aggfunc)

In [ ]:
pivot_products.describe()

### Catégories des produits

Nous allons maintenant créer une variable par catégorie de produit. Nous prendrons les catégories traduites en anglais, plus parlantes que les catégories en portugais. Pour chaque catégorie, nous renseignerons la proportion de produits de cette catégorie acheté par chaque client.

In [ ]:
# Add the English translation of the product category
df = df.merge(right=df_translation, how="inner",
              on="product_category_name", copy=False)

# Create a feature by category
list_products_cat = df["product_category_name_english"].unique().tolist()
aggfunc = {"product_id": 'count'}
for c in list_products_cat:
    cond = df["product_category_name_english"]==c
    df[c] = 1
    df[c] = df[c].where(cond, 0)
    aggfunc[c] = sum

# Create the pivot table
values = ["product_id"] + list_products_cat
pivot_cat = pd.pivot_table(df, index="customer_unique_id",
                           values=values, aggfunc=aggfunc)
# Calculate the proportions
for c in list_products_cat:
    pivot_cat[c] /= pivot_cat["product_id"]
pivot_cat = pivot_cat.drop(columns=["product_id"])

In [ ]:
pivot_cat.describe()

### Variables liées aux commentaires

Nous traiterons maintenant les variables liées aux commentaires. Pour chaque client, nous calculerons les variables suivantes :
- nombre moyens de commentaires par transactions
- score moyen
- temps de réponse moyen

In [ ]:
# Create a new dataframe with all required data
df = pd.merge(df_orders, df_order_reviews, how="inner",
              on="order_id", copy=True)
df = df.merge(right=df_customers, how="inner",
              on="customer_id", copy=False)
# Convert date features into 'datetime' type
df["review_creation_date"] = pd.to_datetime(df["review_creation_date"])
df["review_answer_timestamp"] = pd.to_datetime(df["review_answer_timestamp"])
# Calculate the time to answer in days
df["review_answer_timedelta"] = df["review_answer_timestamp"] - df["review_creation_date"]
df["review_answer_timedelta"] /= pd.to_timedelta(1, unit="D")

values = ["order_id",
          "review_id",
          "review_score",
          "review_answer_timedelta"]
aggfunc = {"order_id": pd.Series.nunique,
           "review_id": 'count',
           "review_score": 'mean',
           "review_answer_timedelta": 'mean'}

pivot_reviews = (pd.pivot_table(df, index="customer_unique_id",
                               values=values, aggfunc=aggfunc)
                  .rename(columns={"order_id": "n_orders",
                                   "review_id": "order_n_reviews"})
                  .assign(order_n_reviews=lambda df: df["order_n_reviews"]/df["n_orders"])
                  .drop(columns=["n_orders"]))

In [ ]:
pivot_reviews.describe()

### Moyens de paiement

Il pourrait être utile de voir le moyen de paiement privilégié par chaque client. Pour cela nous allons créer une variable par moyen de paiement possible, et nous renseignerons la proportion du montant total payé par ce moyen de paiement.

In [ ]:
# Create a new dataframe with all required data
df = pd.merge(df_orders, df_order_payments, how="inner",
              on="order_id", copy=True)
df = df.merge(right=df_customers, how="inner",
              on="customer_id", copy=False)

# Create a feature by type of payment
list_payment_type = df["payment_type"].unique().tolist()
aggfunc = {"payment_value": sum}
for c in list_payment_type:
    cond = df["payment_type"]==c
    df[c] = df["payment_value"]
    df[c] = df[c].where(cond, 0)
    aggfunc[c] = sum

# Create the pivot table
values = ["payment_value"] + list_payment_type
pivot_payment = pd.pivot_table(df, index="customer_unique_id",
                           values=values, aggfunc=aggfunc)
# Calculate the proportions
for c in list_payment_type:
    pivot_payment[c] /= pivot_payment["payment_value"]
pivot_payment = pivot_payment.drop(columns=["payment_value", "not_defined"])

In [ ]:
pivot_payment.describe()

### Les plus gros vendeurs

Nous allons créer une variable pour chacun des 5 plus gros vendeurs. Nous renseignerons ces variables par le pourcentage des produits achetés auprès de ces vendeurs par rapport au total des produits achetés.

In [ ]:
# Get the top five of sellers
top_five_sellers = df_order_items['seller_id'].value_counts()[:5].index.tolist()

# Create a new dataframe with all required data
df = pd.merge(df_orders, df_order_items, how="inner",
              on="order_id", copy=True)
df = df.merge(right=df_customers, how="inner",
                on="customer_id", copy=False)

# Create a feature by top sellers
aggfunc = {"product_id": 'count'}
for c in top_five_sellers:
    cond = df["seller_id"]==c
    df[c] = 1
    df[c] = df[c].where(cond, 0)
    aggfunc[c] = sum

# Create the pivot table
values = ["product_id"] + top_five_sellers
pivot_top_sellers = pd.pivot_table(df, index="customer_unique_id",
                                   values=values, aggfunc=aggfunc)
# Calculate the proportions
for c in top_five_sellers:
    pivot_top_sellers[c] /= pivot_top_sellers["product_id"]
pivot_top_sellers = pivot_top_sellers.drop(columns=["product_id"])  

In [ ]:
pivot_top_sellers.describe()

### États

Les dernière variables que nous allons créer sont les état ou vivent les clients. Il y aura une variable par état possible, avec pour valeur :
- **1** si le client y vit
- **0** sinon.

C'est ce qu'on appelle un **one hot encoding**. Pour cela, nous utiliserons la classe `OneHotEncoder` de *sklearn.preprocessing*.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create the features
df = df_customers.copy()
cat_encoder = OneHotEncoder()
states_1hot = cat_encoder.fit_transform(df['customer_state'].values.reshape(-1, 1))
list_states = cat_encoder.categories_[0].tolist()
df_1hot = pd.DataFrame(states_1hot.toarray(),
                       columns=list_states,
                       index=df.index)
df = pd.concat([df, df_1hot], axis=1)

# Group by unique customer
values = list_states
pivot_states = pd.pivot_table(df, index="customer_unique_id",
                              values=values, aggfunc=np.mean)

### Fusion des pivot tables

Nous allons fusionner toutes les tables créées précédemment afin d'obtenir une table unique.

In [ ]:
from functools import reduce

list_pivot_orders = [pivot_orders,
                     pivot_products,
                     pivot_cat,
                     pivot_reviews,
                     pivot_payment,
                     pivot_top_sellers,
                     pivot_states]
# Merge all DataFrame
data = reduce(lambda left, right: pd.merge(left, right, how="inner",
                                           on="customer_unique_id"),
              list_pivot_orders)
print(data.shape)

## Export des données

In [ ]:
# export in csv format the cleaned/transformed datasets
data.to_csv("data/data_cleaned.csv")